<a href="https://colab.research.google.com/github/JaiSuryaPrabu/deep_learning/blob/main/2_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creation of data

In [ ]:
from sklearn.datasets import make_circles

X,y = make_circles(n_samples=1000,noise=0.35,random_state=42)
X[:5],y[:5]

(array([[ 0.56161627,  0.47476109],
        [-0.40213357,  0.40018308],
        [-1.04686044,  0.95211341],
        [-0.96020855,  0.36244147],
        [ 0.49652102, -0.86811738]]),
 array([1, 1, 1, 1, 0]))

## Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

x_train.shape,y_train.shape

((800, 2), (800,))

## To tensors

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

x_train = torch.from_numpy(x_train).type(torch.float).to(device)
x_test = torch.from_numpy(x_test).type(torch.float).to(device)
y_train = torch.from_numpy(y_train).type(torch.float).to(device)
y_test = torch.from_numpy(y_test).type(torch.float).to(device)

x_train.size(),y_train.size(),x_test.size(),y_test.size()

(torch.Size([800, 2]),
 torch.Size([800]),
 torch.Size([200, 2]),
 torch.Size([200]))

# Model

## Building

In [ ]:
import torch.nn as nn

class classifier(nn.Module):
    def __init__(self):
        super().__init__()

        # parameters
        self.layer1 = nn.Linear(in_features=2,out_features=10)
        self.layer2 = nn.Linear(in_features=10,out_features=10)
        self.layer3 = nn.Linear(in_features=10,out_features=1)

    def forward(self,input):
        return self.layer3(self.layer2(self.layer1(input)))

In [ ]:
model = classifier().to(device)
model

classifier(
  (layer1): Linear(in_features=2, out_features=10, bias=True)
  (layer2): Linear(in_features=10, out_features=10, bias=True)
  (layer3): Linear(in_features=10, out_features=1, bias=True)
)

# Loss and Optimizer

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.1)

## Training and Testing phase

In [ ]:
epochs = 100

for epoch in range(epochs):
    # training mode
    model.train()

    y_preds = torch.round(torch.sigmoid(model(x_train).squeeze()))

    # loss
    loss = loss_fn(y_preds,y_train)

    # zero grad
    optimizer.zero_grad()

    # backward
    loss.backward()

    # take a step
    optimizer.step()

    # testing mode
    model.eval()
    with torch.inference_mode():
        test_pred = torch.round(torch.sigmoid(model(x_test).squeeze()))

        test_loss = loss_fn(test_pred,y_test)

    if epoch % 10 == 0:
        print(f"Epoch {epoch} Training loss {loss} Testing loss {test_loss}")

Epoch 0 Training loss 0.7111011147499084 Testing loss 0.6924488544464111
Epoch 10 Training loss 0.7111011147499084 Testing loss 0.6924488544464111
Epoch 20 Training loss 0.7111011147499084 Testing loss 0.6924488544464111
Epoch 30 Training loss 0.7111011147499084 Testing loss 0.6924488544464111
Epoch 40 Training loss 0.7111011147499084 Testing loss 0.6924488544464111
Epoch 50 Training loss 0.7111011147499084 Testing loss 0.6924488544464111
Epoch 60 Training loss 0.7111011147499084 Testing loss 0.6924488544464111
Epoch 70 Training loss 0.7111011147499084 Testing loss 0.6924488544464111
Epoch 80 Training loss 0.7111011147499084 Testing loss 0.6924488544464111
Epoch 90 Training loss 0.7111011147499084 Testing loss 0.6924488544464111
